In [130]:
import mysql.connector

In [131]:
''''
Host: sql7.freesqldatabase.com
Database name: sql7720700
Database user: sql7720700
Database password: LbMR8S6Jb8
Port number: 3306
'''
conn = mysql.connector.connect(host = "sql7.freesqldatabase.com",
                                user = "sql7720700",
                                password = "LbMR8S6Jb8",
                                database = "sql7720700",
                                port = 3306)
conn

In [132]:
cursor = conn.cursor()
cursor

/* find the number of availalbe copies of the book (Dracula)      */


In [133]:
cursor.execute('''
SELECT b.BookID, b.Title
FROM Loans AS l
LEFT JOIN Books AS b ON b.BookID = l.BookID
WHERE b.Title = 'Dracula' AND b.BookID != (
select b.BookID from Loans as l
inner join Books as b on b.BookID = l.BookID
where Title = 'Dracula' and ReturnedDate is null
)
GROUP BY b.BookID;
''')

In [134]:
num_ava_of_DraculaBook = cursor.fetchall()
num_ava_of_DraculaBook

[(12, 'Dracula'), (60, 'Dracula')]

/* check total copies of the book */


In [135]:
cursor.execute('''
select Title, count(BookID) as numbers_0f_copies from Books
where Title = 'Dracula';
''')

In [136]:
TotalCopiesOf_Dracula = cursor.fetchall()
TotalCopiesOf_Dracula

[('Dracula', 3)]

/* current total loans of the book */


In [137]:
cursor.execute('''
SELECT b.Title, count(*) FROM Loans AS l
inner JOIN Books AS b ON b.BookID = l.BookID
WHERE l.ReturnedDate IS NULL AND b.title = 'Dracula';
''')

In [138]:
loaned_copies_of_dracula = cursor.fetchall()
loaned_copies_of_dracula

[('Dracula', 1)]

/* total available books of dracula */


In [139]:
cursor.execute('''
SELECT b.BookID, COUNT(l.LoanID) AS loaned_count
FROM Loans AS l
LEFT JOIN Books AS b ON b.BookID = l.bookID
WHERE b.Title = 'Dracula' AND b.BookID != (
select b.BookID from Loans as l
inner join Books as b on b.BookID = l.BookID
where Title = 'Dracula' and ReturnedDate is null
)
GROUP BY b.BookID;
''')

In [140]:
ava_books_of_dracula = cursor.fetchall()
ava_books_of_dracula

[(12, 6), (60, 13)]

/* Add new books to the library                        */


In [141]:
# cursor.execute('''
# insert into Books (BookID, Title, Author, Published, Barcode) values (201,'de7k', 'adel', 2019, 2482001);
# insert into Books (BookID, Title, Author, Published, Barcode) values (202,'PYTHON', 'mostafa saad', 2000, 1552000);

# insert into Books (BookID, Title, Author, Published, Barcode) values
# (203, 'journey to the center of the earth', 'Ahmed Ayman', 1999, 2060708),
# (204, 'once upon time','abdelziz', 1998, 45658978),
# (205, 'venom', 'marvel', 1888, 10397850);
# ''')

In [142]:
cursor.execute('''
select * from Books
where BookID = 201;
''')

In [143]:
show_insert1= cursor.fetchall()
show_insert1

[(201, 'de7k', 'adel', 2019, 2482001)]

Check out Books: books(4043822646, 2855934983) whose patron_email(jvaan@wisdompets.com), loandate=2020-08-25, duedate=2020-09-08, loanid=by_your_choice

In [144]:
# cursor.execute('''
# insert into Loans (LoanID, BookID, PatronID, LoanDate, DueDate) values
# (5000,
# (select BookID from Books where Barcode = 2855934983),
# (select PatronID from Patrons where Email = 'jvaan@wisdompets.com'),
# '2020-08-25','2020-09-08'),
# (5001,
# (select BookID from Books where Barcode = 4043822646),
# (select PatronID from Patrons where Email = 'jvaan@wisdompets.com'),
# '2020-08-25','2020-09-08');
# ''')

In [145]:
cursor.execute('''
SELECT * from Loans
where LoanID in (5001,5000);
''')

In [146]:
show_inser2 = cursor.fetchall()
show_inser2

[]

Check books for Due back                             
generate a report of books due back on July 13, 2020 
with patron contact information   

In [147]:
cursor.execute('''
SELECT l.*, Title, p.* FROM Loans AS l
LEFT JOIN Books AS b ON b.BookID = l.BookID
inner join Patrons as p on p.PatronID = l.PatronID
WHERE l.ReturnedDate IS NULL and l.DueDate = '2020-07-13';

''')

In [148]:
report_of_patrons = cursor.fetchall()
report_of_patrons

[(1996,
  10,
  99,
  '2020-06-29',
  '2020-07-13',
  None,
  'The Count of Monte Cristo',
  'Kala',
  'Simonian',
  'ksimonian2q@landonhotel.com',
  99),
 (1997,
  155,
  47,
  '2020-06-29',
  '2020-07-13',
  None,
  'The Prophet',
  'Lydie',
  'Awmack',
  'lawmack1a@samoca.org',
  47),
 (1998,
  169,
  18,
  '2020-06-29',
  '2020-07-13',
  None,
  'Second Treatise of Government',
  'Anthe',
  'Dinjes',
  'adinjesh@samoca.org',
  18),
 (1999,
  73,
  59,
  '2020-06-29',
  '2020-07-13',
  None,
  'Dracula',
  'Cornelia',
  'Koppe',
  'ckoppe1m@landonhotel.com',
  59)]

In [149]:
report_of_patrons[0][1]

10

In [150]:
with open ("Patrons data.csv", mode = 'w', encoding='utf-8') as fd:
    fd.write("LoanID, BookID, LoanDate, DueDate, ReturnedDate, Title, FirstName, LastName, Email, PatronID \n")
    for i in range(len(report_of_patrons)):
        fd.write(f'\"{report_of_patrons[i][0]}\", \"{report_of_patrons[i][1]}\", \"{report_of_patrons[i][2]}\", \"{report_of_patrons[i][3]}\", \"{report_of_patrons[i][4]}\", \"{report_of_patrons[i][5]}\", \"{report_of_patrons[i][6]}\", \"{report_of_patrons[i][7]}\", \"{report_of_patrons[i][8]}\", \"{report_of_patrons[i][9]}\", \"{report_of_patrons[i][10]}\" \n')


In [151]:
import pandas as pd
df = pd.read_csv("Patrons data.csv")
df

,LoanID,BookID,LoanDate,DueDate,ReturnedDate,Title,FirstName,LastName,Email,PatronID
1996,"""10""","""99""","""2020-06-29""","""2020-07-13""","""None""","""The Count of Monte Cristo""","""Kala""","""Simonian""","""ksimonian2q@landonhotel.com""","""99"""
1997,"""155""","""47""","""2020-06-29""","""2020-07-13""","""None""","""The Prophet""","""Lydie""","""Awmack""","""lawmack1a@samoca.org""","""47"""
1998,"""169""","""18""","""2020-06-29""","""2020-07-13""","""None""","""Second Treatise of Government""","""Anthe""","""Dinjes""","""adinjesh@samoca.org""","""18"""
1999,"""73""","""59""","""2020-06-29""","""2020-07-13""","""None""","""Dracula""","""Cornelia""","""Koppe""","""ckoppe1m@landonhotel.com""","""59"""


/* Return books to the library (which have barcode=6435968624) and return this book at this date(2020-07-05)                    */


In [152]:
cursor.execute('''
UPDATE Loans
SET ReturnedDate = '2020-07-05'
WHERE BookID = (
SELECT BookID FROM Books
WHERE Barcode = '6435968624'AND ReturnedDate IS NULL);
''')

In [153]:
cursor.execute('''
select * from Loans
where LoanID = 1991
''')

In [154]:
update_data1 = cursor.fetchall()
update_data1

[(1991, 105, 65, '2020-06-25', '2020-07-09', '2020-07-05')]

Encourage Patrons to check out books                */
/* generate a report of showing 10 patrons who have
checked out the fewest books.       

In [155]:
cursor.execute('''
select p.PatronID, p.FirstName, p.Email, count(LoanID)as checke_out_num from Books as b
inner join Loans as l on l.BookID = b.BookID
inner join Patrons as p on p.PatronID = l.PatronID
group by PatronID
order by checke_out_num
limit 10;
''')

In [156]:
topten_patrons = cursor.fetchall()
topten_patrons

[(83, 'Caril', 'cmatejic2a@orangevalleycaa.org', 9),
 (69, 'Alaster', 'aruggles1w@red30design.com', 9),
 (73, 'Merle', 'msukbhans20@orangevalleycaa.org', 11),
 (94, 'Curr', 'cfittall2l@orangevalleycaa.org', 12),
 (64, 'Norby', 'nsleight1r@kinetecoinc.com', 12),
 (50, 'Jack', 'jvaan@wisdompets.com', 13),
 (7, 'Cammi', 'ckynett6@orangevalleycaa.org', 14),
 (16, 'Carla', 'cswinfenf@kinetecoinc.com', 14),
 (3, 'Aleen', 'afasey2@kinetecoinc.com', 14),
 (12, 'Orton', 'ostaversb@orangevalleycaa.org', 14)]

In [157]:
with open('patrons_report.csv', mode='w', encoding='utf-8') as fd:
    fd.write('PatronsID, FirstName, Email, checked out number \n')
    for i in range(len(topten_patrons)):
        fd.write(f'\"{topten_patrons[i][0]}\", \"{topten_patrons[i][1]}\", \"{topten_patrons[i][2]}\", \"{topten_patrons[i][3]}\" \n')

In [158]:
fd1 = pd.read_csv('patrons_report.csv')
fd1

,PatronsID,FirstName,Email,checked out number
0,83,"""Caril""","""cmatejic2a@orangevalleycaa.org""","""9"""
1,69,"""Alaster""","""aruggles1w@red30design.com""","""9"""
2,73,"""Merle""","""msukbhans20@orangevalleycaa.org""","""11"""
3,94,"""Curr""","""cfittall2l@orangevalleycaa.org""","""12"""
4,64,"""Norby""","""nsleight1r@kinetecoinc.com""","""12"""
5,50,"""Jack""","""jvaan@wisdompets.com""","""13"""
6,7,"""Cammi""","""ckynett6@orangevalleycaa.org""","""14"""
7,16,"""Carla""","""cswinfenf@kinetecoinc.com""","""14"""
8,3,"""Aleen""","""afasey2@kinetecoinc.com""","""14"""
9,12,"""Orton""","""ostaversb@orangevalleycaa.org""","""14"""


Find books to feature for an event                  
 create a list of books from 1890s that are
 currently available      

In [159]:
cursor.execute('''
select b.Title, count(b.BookID) as NumbersOfCopies from Books as b
left join Loans as l on l.BookID = b.BookID
where Published = 1890 and ReturnedDate is not null
group by b.Title
order by NumbersOfCopies DESC;
''')

In [160]:
books_data = cursor.fetchall()
books_data

[('The Picture of Dorian Gray', 39),
 ('An Occurrence at Owl Creek Bridge', 32),
 ('The Sign of the Four', 25)]

In [161]:
with open('books data.csv', mode='w', encoding= 'utf-8') as fd:
    fd.write('Books Title, NumbersOfCopies \n')
    for i in range(len(books_data)):
        fd.write(f'\"{books_data[i][0]}\", \"{books_data[i][1]}\" \n')

In [162]:
fd2 = pd.read_csv('books data.csv')
fd2

,Books Title,NumbersOfCopies
0,The Picture of Dorian Gray,"""39"""
1,An Occurrence at Owl Creek Bridge,"""32"""
2,The Sign of the Four,"""25"""


Book Statistics 
/* create a report to show how many books were 
published each year.        

In [163]:
cursor.execute('''
SELECT Published, COUNT(DISTINCT(Title)) AS TotalNumberOfBooksPublished
FROM Books
GROUP BY Published
ORDER BY TotalNumberOfBooksPublished DESC;
''')

In [164]:
TotalNumberOfBooksPublished = cursor.fetchall()
TotalNumberOfBooksPublished

[(1890, 3),
 (1895, 3),
 (1892, 2),
 (1859, 2),
 (1922, 2),
 (1869, 2),
 (1898, 2),
 (1899, 2),
 (1847, 2),
 (1884, 2),
 (1850, 2),
 (1911, 2),
 (1605, 1),
 (1852, 1),
 (1889, 1),
 (1913, 1),
 (1670, 1),
 (1854, 1),
 (1914, 1),
 (1689, 1),
 (1858, 1),
 (1915, 1),
 (1726, 1),
 (1894, 1),
 (1920, 1),
 (1729, 1),
 (1861, 1),
 (1921, 1),
 (1759, 1),
 (1865, 1),
 (1896, 1),
 (1791, 1),
 (1897, 1),
 (1923, 1),
 (1811, 1),
 (1876, 1),
 (1998, 1),
 (1813, 1),
 (1878, 1),
 (1999, 1),
 (1815, 1),
 (1879, 1),
 (1900, 1),
 (2019, 1),
 (1818, 1),
 (1880, 1),
 (1901, 1),
 (1839, 1),
 (1883, 1),
 (1902, 1),
 (1905, 1),
 (1849, 1),
 (1886, 1),
 (1906, 1),
 (1375, 1),
 (1887, 1),
 (1908, 1),
 (1532, 1),
 (1851, 1),
 (1888, 1)]

 Book Statistics                                           */
/* create a report to show 5 most popular Books to check out */

In [165]:
cursor.execute('''
select b.Title ,count(LoanID) as loan_count from Books as b
inner join Loans as l ON b.BookID = l.BookID
group by b.Title
order by loan_count desc
limit 5;
''')

In [166]:
TopFiveBooks = cursor.fetchall()
TopFiveBooks

[('Frankenstein; Or, The Modern Prometheus', 53),
 ('The Awakening, and Selected Short Stories', 43),
 ('The War of the Worlds', 43),
 ('The Scarlet Letter', 42),
 ('The Picture of Dorian Gray', 39)]

https://github.com/mostafa-s-mostafa/SQL-Project